# Experiments with openai actions
In this notebook, I perform some experimentation with the capabilities of openai's assistants API.  I plan to see how effective it is at extracting locations and property features from a query and using that data to identify which calls to make to reapi

In [43]:

import os
from dotenv import load_dotenv
load_dotenv() # Load in OPENAI_API_KEY, REAPI_API_KEY
from openai import OpenAI
from helpers import wait_on_run, create_thread_and_run, submit_message, get_response
import requests


import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))


## Create new assistant

In [ ]:
client = OpenAI()
assistant = client.beta.assistants.create(
	name="Property Finder",
	instructions="",
	model="gpt-4-1106-preview",
)

## Use existing assistant ID

In [23]:
ASSISTANT_ID="asst_I1A0q0vqDL1Nwx6kpN1lBTxC"
assistant = client.beta.assistants.retrieve(ASSISTANT_ID)

## Initiate Message Thread

In [9]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_lGoYOH0yaHN28e2wg46LoAe1',
 'created_at': 1704831029,
 'metadata': {},
 'object': 'thread'}

## Sending Messages

### Construct the Message

In [11]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)


### Send the Message

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
) # returns a preliminary object immediately and asynchronously calls the api.  Must wait on a response.
run = wait_on_run(run, thread)

### Show AI Response messages in the given thread 
(For simple qa threads with only one propt)

In [21]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
#show_json(messages)
print([msg.content[0].text.value if msg.role == 'assistant' else "" for msg in messages.data])

["Certainly! To solve the equation `3x + 11 = 14` for x, you'll need to isolate x on one side of the equation. Here's how you can do that step-by-step:\n\n1. **Subtract 11 from both sides of the equation:**\n\n   `3x + 11 - 11 = 14 - 11`\n\n   This simplifies to:\n\n   `3x = 3`\n\n2. **Divide both sides of the equation by 3:**\n\n   `(3x) / 3 = 3 / 3`\n\n   This simplifies to:\n\n   `x = 1`\n  \nSo the solution to the equation `3x + 11 = 14` is `x = 1`.", '']


## Exctracting data from the prompt as json

In [36]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(model='gpt-4-1106-preview')

from langchain.chains import create_extraction_chain

schema = {
    "properties": {
				"location": {"type": "string"},
    		"beds": {"type": "integer"},
    		"baths": {"type": "integer"},
				"max_price": {"type": "integer"}
    },
    "required": [],
}

extraction_chain = create_extraction_chain(schema, model)

In [37]:
prompt = "find me a three bed, 2 bath house with a pool and a firepit for about 600,000 in flagler beach, Florida"
extracted_data = extraction_chain.run(prompt)

In [71]:


autocomplete_url = "https://api.realestateapi.com/v2/AutoComplete"

autocomplete_payload = {
    "search": "Flagler beach, Fl"
}
autocomplete_headers = {
    "accept": "application/json",
    "x-user-id": "theo",
    "content-type": "application/json",
	"x-api-key": "COMPASS-cd32-712d-98ae-855716d9f8f2"
}

autocomplete_response = requests.post(autocomplete_url, json=autocomplete_payload, headers=autocomplete_headers)

In [86]:

beds = extracted_data[0]['beds']
baths = extracted_data[0]['baths']
max_price = extracted_data[0]['max_price']
city = json.loads(autocomplete_response.content)['data'][0]['city']
state = json.loads(autocomplete_response.content)['data'][0]['state']
# json.loads(autocomplete_response.content)['data'][0]

property_search_url = "https://api.realestateapi.com/v2/PropertySearch"

property_search_payload = {
		"ids_only": False,
    "obfuscate": False,
    "summary": False,
    "size": 5,
    "city": city,
    "state": state,
		"beds_min": beds,
		"baths_min": baths,
		"mls_listing_price": max_price,
		"mls_listing_price_operator": "lte"

}
property_search_headers = {
    "accept": "application/json",
    "x-user-id": "theo",
    "content-type": "application/json",
		"x-api-key": "COMPASS-cd32-712d-98ae-855716d9f8f2"
}

property_search_response = requests.post(property_search_url, json=property_search_payload, headers=property_search_headers)

In [90]:
json.loads(property_search_response.content)['data']

[{'absenteeOwner': False,
  'address': {'address': '1635 S Flagler Ave, Flagler Beach, Fl 32136',
   'city': 'Flagler Beach',
   'county': 'Flagler',
   'state': 'FL',
   'street': '1635 S Flagler Ave',
   'zip': '32136'},
  'adjustableRate': False,
  'airConditioningAvailable': False,
  'apn': '18-12-32-2750-00120-0150',
  'assessedImprovementValue': 230203,
  'assessedLandValue': 134000,
  'assessedValue': 364203,
  'assumable': False,
  'auction': False,
  'auctionDate': None,
  'basement': False,
  'bathrooms': 2,
  'bedrooms': 3,
  'cashBuyer': True,
  'corporateOwned': False,
  'death': False,
  'deck': False,
  'deckArea': 0,
  'documentType': 'Intrafamily Transfer',
  'documentTypeCode': 'DTIT',
  'equity': False,
  'equityPercent': 100,
  'estimatedEquity': 439616,
  'estimatedValue': 439616,
  'floodZone': True,
  'floodZoneDescription': '0.2',
  'floodZoneType': 'X',
  'foreclosure': False,
  'forSale': False,
  'freeClear': False,
  'garage': False,
  'highEquity': True,
  